In [1]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression #Logistic(Regression)Classifier
from sklearn.tree import DecisionTreeClassifier #Decision Tree
from sklearn.svm import SVC #Support Vector Machine
from sklearn.naive_bayes import GaussianNB #Naive Bayesian
from sklearn.neighbors import KNeighborsClassifier #K Nearest Neighbor
from sklearn.ensemble import RandomForestClassifier #Random Forest
from sklearn.ensemble import GradientBoostingClassifier #Gradient Boosing
from sklearn.neural_network import MLPClassifier #Neural Network
from sklearn.metrics import accuracy_score
from sklearn import model_selection

class FedLearning(object):
    
    #Loading iris data and shuffling for spliting data to clients
    def __init__(self, obj_data):
        self.data = obj_data        
        df = pd.DataFrame(data = self.data.data, columns = self.data.feature_names)
        df['target'] = self.data.target
        df['target'] = df['target'].map({0:"setosa", 1:"versicolor", 2:"virginica"})
        df = shuffle(df)
        self.x_data = df.iloc[:, :-1]
        self.y_data = df.iloc[:, [-1]]
    
    #Spliting data by the number of clients
    def client_data(self, num):
        client = [];client_ans = []
        split_loc = len(self.x_data)//num
        for i in range(num):
            temp_x = self.x_data.iloc[split_loc*i:split_loc*(i+1), :]
            temp_y = self.y_data.iloc[split_loc*i:split_loc*(i+1), :]
            client.append(temp_x)
            client_ans.append(temp_y)
        print(client_ans, len(client_ans[0]))
        return client, client_ans
    
    #initialize weight of model
    def weight_ini(self):
        '''
        LR = LogisticRegression()
        LR.fit(x_data, y_data.values.ravel())
        y_pred = LR.predict(x_data)
        print("LR's Accuracy is", accuracy_score(y_data, y_pred))
        coef = LR.coef_
        intercept = LR.intercept_
        print(coef, intercept)
        
        1. data split : the number of clients equals three
        setosa 50
        versicolor 50
        virginica 50

        2. weight initialization 

        3. client별로 split된 data가지고 학습
        (이때 split된 data에 대해서 train set과 test set을 나눔)

        50 50 50   m번 학습
        40 40 40 => training(40개 data중 n개 sampling 해서 학습)


        맨 마지막에
        10 10 10 => test 
        '''
                      
        return
    
iris = load_iris()
FL = FedLearning(iris)
a = FL.client_data(3)
#print(a)

[         target
142   virginica
12       setosa
8        setosa
34       setosa
28       setosa
147   virginica
30       setosa
89   versicolor
60   versicolor
95   versicolor
44       setosa
33       setosa
140   virginica
81   versicolor
126   virginica
71   versicolor
128   virginica
2        setosa
131   virginica
6        setosa
0        setosa
59   versicolor
149   virginica
32       setosa
36       setosa
52   versicolor
69   versicolor
50   versicolor
99   versicolor
24       setosa
93   versicolor
137   virginica
15       setosa
97   versicolor
67   versicolor
19       setosa
73   versicolor
66   versicolor
10       setosa
27       setosa
123   virginica
103   virginica
49       setosa
94   versicolor
101   virginica
55   versicolor
124   virginica
83   versicolor
54   versicolor
141   virginica,          target
138   virginica
115   virginica
92   versicolor
40       setosa
130   virginica
31       setosa
121   virginica
35       setosa
120   virginica
22       setosa
113   